In [ ]:
import torch
mlp = torch.load('./model_full.pth')
for layer in mlp.layers:
    if (isinstance(layer, torch.nn.Linear)):
        w=layer.weight
        b=layer.bias
        U,S,V=torch.linalg.svd(w)
        nU=U.shape[0]
        mU=U.shape[1]
        nS=S.shape[0]
        mS=S.shape[1]
        nV=V.shape[0]
        mV=V.shape[1]
        newlinear1=torch.nn.Linear(nU,mU)
        newlinear2=torch.nn.Linear(nS,mS)
        newlinear3=torch.nn.Linear(nV,mV)
        newlinear1.weight=U
        newlinear2.weight=S
        newlinear3.weight=V

